In [6]:
from sage.all import *
import numpy as np

In [7]:
def assign_simplex_values(simplex, func):
    vals = np.array([func(*v) for v in simplex])  # try using np for array; later check if better
    idx = np.argsort(vals)
    srt = np.array(simplex)[idx]
    v0, v1, v2 = srt         # best, good, worst
    f0, f1, f2 = vals[idx]
    return vector(v0), vector(v1), vector(v2), f0, f1, f2

In [8]:
def nelder_mead_search(func, init_simplex, tol=1e-6, max_iter=500):
    simplex = [vector(v) for v in init_simplex]
    for _ in range(max_iter):
        B, G, W, fB, fG, fW = assign_simplex_values(simplex, func)
        M = (B + G) / 2
        R = 2*M - W
        fR = func(*R)
        if fR < fB:
            E = 2*R - M
            fE = func(*E)
            W = E if fE < fB else R
        else:
            if fR < fG:
                W = R
            else:
                C1 = (M + R) / 2
                C2 = (M + W) / 2
                C = C1 if func(*C1) < func(*C2) else C2
                if func(*C) > fW:
                    G = (B + G) / 2
                    W = (B + W) / 2
                else:
                    W = C
        simplex = [B, G, W]
        if (W - B).norm() < tol:
            break
    return tuple(B), func(*B)

In [9]:
f = lambda a, b: 6*a**2 - 6*a*b + 2*b**2 - a - 2*b

S = [(-10, 10), (-5, -7), (10.0, -10.0)]

opt_pt, opt_val = nelder_mead_search(f, S)

In [10]:
opt_pt, opt_val = nelder_mead_search(f, S)

print("Optimal Point:", opt_pt)
print("Optimal Value:", opt_val)

Optimal Point: (1.3333342120878675, 2.5000005184220644)
Optimal Value: -3.166666666664225


In [11]:
x, y = var('x y')
f = lambda x, y: 25*x**2 - 12*x**4 - 6*x*y + 25*y**2 - 24*x**2*y**2 - 12*y**4

In [12]:
zmin, zmax, h = -2.0, 10.0, 0.4
nc = int(round((zmax - zmin) / h))
levels = [zmin + i*h for i in range(nc + 1)]

cont = contour_plot(f(x,y), 
                    (x,-0.8,0.8), 
                    (y,-0.6,0.6),
                    fill=False, linewidths=1, contours=levels, cmap='cool')
S = [vector(v) for v in [(-0.6,-0.6), (-0.4,-0.6), (-0.6,-0.4)]]
n, m = len(S[0]), len(S)
frame = cont + point(S) + polygon(S, fill=False, color='black')

In [13]:
frames = [frame]
for _ in range(20):
    vals = [f(v[0], v[1]) for v in S]
    iLo, iHi = min(range(m), key=lambda i: vals[i]), max(range(m), key=lambda i: vals[i])

    sv = vector([sum(v[j] for v in S) for j in range(n)])
    d = (-(m+1)*S[iHi] + sv)/m

    R = S[iHi] + 2*d
    fR = f(R[0], R[1])
    if fR <= vals[iLo]:
        E = S[iHi] + d
        fE = f(E[0], E[1])
        S[iHi] = E if fE <= vals[iLo] else R
    else:
        if fR <= vals[iHi]:
            S[iHi] = R
        else:
            C1 = (S[iHi] + R)/2
            C2 = (S[iHi] + S[iLo])/2
            C = C1 if f(C1[0],C1[1]) < f(C2[0],C2[1]) else C2
            if f(C[0],C[1]) <= vals[iHi]:
                S[iHi] = C
            else:
                for i in range(m):
                    if i != iLo:
                        S[i] = (S[i] + S[iLo])/2

    frames.append(cont + point(S) + polygon(S, fill=False, color='black'))

In [14]:
anime = animate(frames, figsize=[6,5], xmin=-0.8, xmax=0.8, ymin=-0.6, ymax=0.6)
anime.show()

Animation with 21 frames


## Higher dimensions try

In [3]:
def assign_simplex_pts(simplex, func):
    vals = [float(func(*pt)) for pt in simplex]
    idx = sorted(range(len(vals)), key=lambda i: vals[i])
    pts = [simplex[i] for i in idx]
    vals = [vals[i] for i in idx]           # p_1 would be the best and the other will be proceeding far away
    return pts, vals             # these are sorted wrt vals

In [4]:
def nelder_mead(func, init, tol=1e-6, max_iter=200):
    # init : list of n+1 points (each point is an iterable of length n)
    n = len(init[0])             # dimension
    m = n + 1                    # simplex size
    simplex = [vector(pt) for pt in init]

    for cycle in range(max_iter):
        simplex, fvals = assign_simplex_pts(simplex, func)
        p1, p2, *middle, pm = simplex   
        f1, f2, *_, fm = fvals

        centroid = sum(simplex[:-1]) / n

        # reflection
        pr = centroid + (centroid - pm)
        fr = float(func(*pr))

        if fr < f1:
            # expansion
            pe = centroid + 2*(centroid - pm)
            fe = float(func(*pe))
            pm = pe if fe < fr else pr
        elif fr < f2:
            # accept reflection
            pm = pr
        else:
            # contraction
            pc = centroid + 0.5*(pm - centroid)
            fc = float(func(*pc))
            if fc < fm:
                pm = pc
            else:
                # shrink toward best p1
                for i in range(1, m):
                    simplex[i] = p1 + 0.5*(simplex[i] - p1)

        simplex[-1] = pm

        edges = [(simplex[i] - p1).norm() for i in range(1, m)]
        if max(edges) < tol:
            break
            
    best = tuple(simplex[0])
    best_val = float(func(*simplex[0]))
    return best, best_val

In [36]:
w, x, y, z = var('w x y z')
f = lambda x, y, z: (x-2)**2 + (y+1)**2 + (z-3)**2
# w, x, y, z = var('w x y z')
# f(x,y,z) = (x-2)^2 + (y+1)^2 + (z-3)^2
S = [(0,0,0), (1,0,0), (0,1,0), (0,0,1)]
minimizer = nelder_mead(f, S)
minimizer

((2.000000035430465, -1.0000001767497182, 3.000000248917237),
 9.445557161467574e-14)

In [39]:
PsatA, PsatB, Ptarget = 100, 200, 150
f = lambda xA,xB,gA,gB: (xA*gA*PsatA + xB*gB*PsatB - Ptarget)**2   # min should be 0 obvioimport numpy as np
def f_rastrigin4(w,x,y,z):
    xs = np.array([w,x,y,z])
    return 10*4 + np.sum(xs**2 - 10*np.cos(2*np.pi*xs))

init4_rastrigin = [
    (3,3,3,3), (4,3,3,3), (3,4,3,3), (3,3,4,3), (3,3,3,4)
]

S = [(0.5,0.5,1.0,1.0), (0.6,0.4,1.0,1.0), (0.5,0.5,1.2,1.0),(0.5,0.5,1.0,1.2), (0.4,0.6,1.0,1.0)]
raoult = nelder_mead(f,S)
raoult

((0.5, 0.5, 1.0, 1.0), 0.0)

In [5]:
xs = lambda w, x, y, z: [w, x, y, z]   # npize later
f = lambda w, x, y, z: 40 + ((w**2 - 10*np.cos(2*np.pi*w)) +
                             (x**2 - 10*np.cos(2*np.pi*x)) +
                             (y**2 - 10*np.cos(2*np.pi*y)) +
                             (z**2 - 10*np.cos(2*np.pi*z)))
S = [(3,3,3,3), (4,3,3,3), (3,4,3,3), (3,3,4,3), (3,3,3,4)]
minimizer = nelder_mead(f, S)
minimizer

((2.984855702001393, 2.984855438316628, 2.984855520039833, 2.984855749573092),
 35.81840496596862)

In [48]:
f = lambda D, v, mu, L: (32 * mu * v * L) / (D**2) + 0.01*(D-0.05)**2
S = [(0.05, 1.0, 1e-3, 10),
     (0.06, 1.0, 1e-3, 10),
     (0.05, 1.2, 1e-3, 10),
     (0.05, 1.0, 2e-3, 10),
     (0.05, 1.0, 1e-3, 12)]
nelder_mead(f,S)   # mini values XD

((1.4432178513276172e-11,
  12.031043053668334,
  -0.1181207274676812,
  78.12679917516783),
 -1.7057509424274113e+25)